In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import torch
from matplotlib import pyplot as plt
import seaborn as sns
import bitsandbytes as bnb

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoModelForCausalLM, GenerationConfig

import openai
openai.api_key = # your OpenAI key here


from googleapiclient import discovery
import json
import time
API_KEY = # your Perspective key here
client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

path = # your path here

In [ ]:
dataset = # dataset
df = pd.read_csv(path+dataset)

# Importing Seq2Seq models and running the inference

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("declare-lab/flan-alpaca-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("declare-lab/flan-alpaca-xl",
                                              torch_dtype="auto",
                                              device_map="auto",
                                              load_in_8bit=True
                                              )

In [ ]:
prompt_template = "Classify the input text as 'respectful' or 'toxic'.\nInput text: {text}.\nAnswer: "
label_map = {
        "toxic": 'toxic',
        "'toxic'": 'toxic',
        "Toxic": 'toxic',
        "'Toxic'": 'toxic',
        "respectful": 'respectful',
        "'respectful'": 'respectful',
        "Respectful": 'respectful',
        "'Respectful'": 'respectful'
    }

batch_size = 100  # You can adjust this based on your available GPU memory
predicted_labels_1 = []
predicted_labels_2 = []

total_rows = len(df)
num_batches = (total_rows + batch_size - 1) // batch_size

for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, total_rows)

    batch_df = df.iloc[start_idx:end_idx]

    for index, row in batch_df.iterrows():

        text1 = row['Sentence 1']
        text2 = row['Sentence 2']

        prompt = prompt_template.format(text=text1)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids, max_length=20)
        predicted_label_1 = tokenizer.decode(outputs[0], skip_special_tokens=True)
        standardized_label_1 = label_map.get(predicted_label_1, predicted_label_1)
        predicted_labels_1.append(standardized_label_1)

        prompt = prompt_template.format(text=text2)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids, max_length=20)
        predicted_label_2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
        standardized_label_2 = label_map.get(predicted_label_2, predicted_label_2)
        predicted_labels_2.append(standardized_label_2)

    torch.cuda.empty_cache()

In [ ]:
filename = # predictions/dataset_model.csv
ids = df['id'].tolist()

pd.DataFrame({'id':ids, 'results':predicted_labels}).to_csv(path+filename, index=False)

# Importing SeqClassification models and run inference

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r2-target")
model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r2-target",
                                                           torch_dtype="auto",
                                                           device_map="auto").to("cuda")

In [ ]:
batch_size = 100  # You can adjust this based on your available GPU memory
predicted_labels_1 = []
predicted_labels_2 = []

total_rows = len(df)
num_batches = (total_rows + batch_size - 1) // batch_size

for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, total_rows)

    batch_df = df.iloc[start_idx:end_idx]

    for index, row in batch_df.iterrows():

        text1 = row['Sentence 1']
        text2 = row['Sentence 2']

        # sentence 1
        inputs = tokenizer(text1, return_tensors="pt", truncation=True, padding=True).to("cuda")
        with torch.no_grad():
            outputs = model(**inputs)

        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        predicted_label_1 = label_map[predicted_label_idx]
        predicted_labels_1.append(predicted_label_1)

        # sentence 2
        inputs = tokenizer(text2, return_tensors="pt", truncation=True, padding=True).to("cuda")
        with torch.no_grad():
            outputs = model(**inputs)

        predicted_label_idx = torch.argmax(outputs.logits, dim=1).item()
        predicted_label_2 = label_map[predicted_label_idx]
        predicted_labels_2.append(predicted_label_2)

    torch.cuda.empty_cache()

In [ ]:
filename = # predictions/dataset_model.csv
ids = df['id'].tolist()

pd.DataFrame({'id':ids, 'results':predicted_labels}).to_csv(path+filename, index=False)

# Importing OpenAI models and run inference

In [ ]:
def classify_hate_speech(texts, model):

  prompt_template = (
            "Classify the input text as 'respectful' or 'toxic'.\n"
            "Input text: {text}.\n"
            "Answer with only one word, 'respectful' or 'toxic': ")

  responses = []
  for text in tqdm(texts):
    prompt = prompt_template.format(text=text)
    try:
      response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant specialized in text classification."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=10)

      label = response['choices'][0]['message']['content'].strip()
      responses.append(label)
    except Exception as e:
      print(f"Error processing text: {text}\n{e}")
      responses.append("Error")

  return responses

In [ ]:
texts_1 = df['Sentence 1'].tolist()
classifications_1 = classify_hate_speech(texts_1, model="gpt-3.5-turbo")

In [ ]:
texts_2 = df['Sentence 2'].tolist()
classifications_2 = classify_hate_speech(texts_2, model="gpt-3.5-turbo")

In [ ]:
label_map = {
        "toxic": 'toxic',
        "'toxic'": 'toxic',
        "Toxic": 'toxic',
        "'Toxic'": 'toxic',
        "respectful": 'respectful',
        "'respectful'": 'respectful',
        "Respectful": 'respectful',
        "'Respectful'": 'respectful'
    }


predicted_labels_1 = []
for classification in classifications_1:
  standardized_label = label_map.get(classification, classification)
  predicted_labels_1.append(standardized_label)

predicted_labels_2 = []
for classification in classifications_2:
  standardized_label = label_map.get(classification, classification)
  predicted_labels_2.append(standardized_label)

In [ ]:
filename = # predictions/dataset_model.csv
ids = df['id'].tolist()

pd.DataFrame({'id':ids, 'results':predicted_labels}).to_csv(path+filename, index=False)

# Open AI Moderation API

In [ ]:
def classify_hate_speech(texts, model="text-moderation-latest"):
    prompt_template = (
            "Classify the input text as 'respectful' or 'toxic'.\n"
            "Input text: {text}.\n"
            "Answer with only one word, 'respectful' or 'toxic': "
        )

    responses = []
    for text in tqdm(texts):

        prompt = prompt_template.format(text=text)

        try:
            response = openai.Moderation.create(
                model=model,
                input=prompt
                )

            label = response['results'][0]['flagged']
            responses.append(label)
        except Exception as e:
            print(f"Error processing text: {text}\n{e}")
            responses.append("Error")

    return responses

In [ ]:
texts_1 = df['Sentence 1'].tolist()
classifications_1 = classify_hate_speech(texts_1)

In [ ]:
texts_2 = df['Sentence 2'].tolist()
classifications_2 = classify_hate_speech(texts_2)

In [ ]:
label_map = {
    False: "respectful",
    True: "toxic"
    }

predicted_labels_1 = []
for classification in classifications_1:
  standardized_label = label_map.get(classification, classification)
  predicted_labels_1.append(standardized_label)

predicted_labels_2 = []
for classification in classifications_2:
  standardized_label = label_map.get(classification, classification)
  predicted_labels_2.append(standardized_label)

In [ ]:
filename = # predictions/dataset_model.csv
ids = df['id'].tolist()

pd.DataFrame({'id':ids, 'results':predicted_labels}).to_csv(path+filename, index=False)

# Perspective API

In [ ]:
def classify_hate_speech(texts, language='english'):

  responses = []
  for text in tqdm(texts):

    analyze_request = {'comment': { 'text': text },
                       'languages': ['en'],
                       'requestedAttributes': {'TOXICITY': {}}
                       }

    try:
        response = client.comments().analyze(body=analyze_request).execute()
        time.sleep(1)
    except:
        responses.append('respectful')
    else:
        hate_speech_value = response['attributeScores']['TOXICITY']['summaryScore']['value']

        if hate_speech_value > 0.5:
            responses.append('toxic')
        else:
            responses.append('respectful')

  return responses

In [ ]:
texts_1 = df['Sentence 1'].tolist()
classifications_1 = classify_hate_speech(texts_1)

texts_2 = df['Sentence 2'].tolist()
classifications_2 = classify_hate_speech(texts_2)

In [ ]:
filename = # predictions/dataset_model.csv
ids = df['id'].tolist()

pd.DataFrame({'id':ids, 'results':classifications}).to_csv(path+filename, index=False)

# Importing DeepSeek

In [ ]:
model_name = "deepseek-ai/deepseek-llm-7b-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

In [ ]:
prompt_template = "Classify the input text as 'respectful' or 'toxic'.\nInput text: {text}.\nAnswer: "

batch_size = 100  # You can adjust this based on your available GPU memory
predicted_labels_1 = []
predicted_labels_2 = []

total_rows = len(df)
num_batches = (total_rows + batch_size - 1) // batch_size

for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, total_rows)

    batch_df = df.iloc[start_idx:end_idx]

    for index, row in batch_df.iterrows():

        text1 = row['Sentence 1']
        text2 = row['Sentence 2']

        prompt = prompt_template.format(text=text1)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids, max_new_tokens=20)
        generated_tokens = outputs[0][input_ids.shape[1]:]  # decode only the newly generated tokens, ignore the prompt tokens
        predicted_label_1 = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        if 'toxic' in predicted_label_1.lower():
          standardized_label_1 = 'toxic'
        elif 'respectful' in predicted_label_1.lower():
          standardized_label_1 = 'respectful'
        else:
          standardized_label_1 = predicted_label_1

        predicted_labels_1.append(standardized_label_1)


        prompt = prompt_template.format(text=text2)
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids, max_new_tokens=20)
        generated_tokens = outputs[0][input_ids.shape[1]:]  # decode only the newly generated tokens, ignore the prompt tokens
        predicted_label_2 = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        if 'toxic' in predicted_label_2.lower():
          standardized_label_2 = 'toxic'
        elif 'respectful' in predicted_label_2.lower():
          standardized_label_2 = 'respectful'
        else:
          standardized_label_2 = predicted_label_2

        predicted_labels_2.append(standardized_label_2)

    torch.cuda.empty_cache()

In [ ]:
filename = # predictions/dataset_model.csv
ids = df['id'].tolist()

pd.DataFrame({'id':ids, 'results':predicted_labels}).to_csv(path+filename, index=False)